# Selenium
**Selenium** es una _Web Browser Automation Tool_, es decir, permite abrir un buscador y realizar tareas que "imiten" el comportamiento humano (clicks, scrolling, *búsquedas*, etc).  
Link a la documentación de Selinium: [https://selenium-python.readthedocs.io/](https://selenium-python.readthedocs.io/)  
Link a la dicumentación y descarga de chromedriver: [https://chromedriver.chromium.org/home](https://chromedriver.chromium.org/home)

In [1]:
# !pip install selenium

In [2]:
# !apt-get install -y chromium-browser

In [3]:
""" !apt-get update
# !apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') """

" !apt-get update\n# !apt install chromium-chromedriver\n!cp /usr/lib/chromium-browser/chromedriver /usr/bin\nimport sys\nsys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') "

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm

import pandas as pd
import time

In [5]:
from selenium import webdriver

def configure_driver():
    # Add additional Options to the webdriver
    options = webdriver.ChromeOptions()
    # add the argument and make the browser Headless.
    # options.add_argument('--headless')
    # options.add_argument('--no-sandbox') # only colab
    # options.add_argument('--disable-dev-shm-usage') # only colab
    # Handle the service
    # service = Service(executable_path=r'/usr/bin/chromedriver')
    # Instantiate the Webdriver: Mention the executable path of the webdriver you have downloaded
    driver = webdriver.Chrome(options=options)

    #driver = webdriver.Firefox()
    return driver

In [ ]:
def getText(driver, total_clicks):
    # https://www.pluralsight.com/search?q=web%20scraping&categories=course
    # Step 1: Go to pluralsight.com, category section with selected search keyword
    # driver.get("https://www.pluralsight.com/search?q={search_keyword}&categories=course")
    # driver.get("https://booknet.com/es/reader/una-pequena-promesa-b425858")
    # driver.get("https://booknet.com/es/reader/un-amor-para-april-b425666")
    driver.get("https://www.allfreenovel.com/Page/Story/88872/page-1-of-Picked-by-the-Billionaire--Covington-Billionaires-/1/154")

    timeout = 3
    count = 0
    loop_obj = tqdm(range(total_clicks))

    data = []
    for k in loop_obj:
        # loop_obj.set_description(f"Epoch: {k + 1}")
        loop_obj.set_postfix_str(f"Count error: {count}")

        # wait for the element to load
        try:
            WebDriverWait(driver, timeout*10).until(lambda s: s.find_element(By.CLASS_NAME, "container").is_displayed())
        except TimeoutException:
            print("Timeout :(")
            return None
        
        # Step 2: Create a parse tree of page sources after searching
        soup = BeautifulSoup(driver.page_source, "lxml")

        # Step 3: Iterate over the search result and fetch the course
        # content = soup.select("div.px-2 div.my-2 div div")

        content = soup.find_all('p', {"class": "storyText"})
        # print(content)
        for course_page in content:
            data.append(course_page.text)
        
        try:
            element = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='btn btn-sm btn-info']")))
            element.click()
            count = 0
        except TimeoutException as r:
            count += 1
            if count > 5:
                break
        time.sleep(timeout)
    return data

total_clicks = 500
# create the driver object.
driver = configure_driver()
data = getText(driver, total_clicks)

100%|██████████| 1/1 [00:00<00:00, 10.61it/s, Count error: 0]

[<p><strong>Title</strong>: A Summer journey in the West</p>, <p><strong>Author</strong>: Mrs. Eliza R. Steele</p>, <p><strong>Release date</strong>: September 25, 2023 [eBook #71725]</p>, <p><strong>Language</strong>: English</p>, <p><strong>Original publication</strong>: NYC: John. S. Taylor, and Co, 1841</p>, <p><strong>Credits</strong>: Chuck Greif and the Online Distributed Proofreading Team at https://www.pgdp.net (This file was produced from images generously made available by The Internet Archive)</p>, <p class="toc">
<a class="pginternal" href="#LETTER_I">LETTER I.</a>,
<a class="pginternal" href="#LETTER_II"> II.</a>,
<a class="pginternal" href="#LETTER_III"> III.</a>,
<a class="pginternal" href="#LETTER_IV"> IV.</a>,
<a class="pginternal" href="#LETTER_V"> V.</a>,
<a class="pginternal" href="#LETTER_VI"> VI.</a>,
<a class="pginternal" href="#LETTER_VII"> VII.</a>,
<a class="pginternal" href="#LETTER_VIII"> VIII.</a>,
<a class="pginternal" href="#LETTER_IX"> IX.</a>,
<a class

In [7]:
# close the driver.
driver.close()

In [86]:
data

[]

In [66]:
file = open("book1.txt", "w")
for line in tqdm(data):
    file.write("{} \n".format(line))
file.close()

100%|██████████| 1271/1271 [00:00<00:00, 909099.66it/s]


---

In [76]:
import nltk
nltk.download("punkt")

file = open("book1.txt")
book = file.read()

[nltk_data] Downloading package punkt to /home/win7/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [77]:
book_sentences = nltk.sent_tokenize(book)
len(book_sentences)
book_sentences[1]

'Yo sonrío como la tía tonta y babosa que soy por sus sobrinos de un año que están aprendiendo a caminar.'

In [78]:
porter = nltk.stem.PorterStemmer()

In [79]:
porter.stem("aprendiendo") # get same root

'aprendiendo'